In [1]:
#General stuff
import time
import datetime
import glob
import re
import pandas as pd
from collections import Counter
import progressbar
import jellyfish
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
import numpy as np
import statsmodels.formula.api as smf
from collections import Counter
from itertools import chain

## Pulling in all the files

In [2]:
df = pd.read_csv('data/df_final_harm.csv')

## I want to look for repetitions "sozialrechtliche Abteilungen"
And I'm working with the years, as the whole long string consisting of 15'000 verdicts is just too long. The Kernel dies regularly. Which really isn't much fun.

In [3]:
df_sozrecht = df[df['recht_harm'] == 'Zivilrecht']
sozrecht = df_sozrecht[df_sozrecht['Language'] == 'Fr']['Text-Nummer']

Splitting up the lists.

In [4]:
l = list(sozrecht)
n = 1000
txt_lsts = [l[i:i + n] for i in range(0, len(l), n)]

In [5]:
df_ = pd.DataFrame(columns=['index', 0])
bar = progressbar.ProgressBar()

for txt_lst, i in zip(txt_lsts, bar(range(len(txt_lsts)))):

    long_str = ''
    
    for txt in txt_lst:
    
        file = open('txt_files/' + txt, 'r')
        text = file.read()
        #Making sure the line breaks '/n', 'xa0' and double spaces are removed
        text = text.replace('\n', ' ').replace('\xa0', '').replace('  ', ' ')
        #Removing these automated notifications
        text = text.replace('Wichtiger Hinweis:Diese Website wird in älteren Versionen von Netscape ohne graphische Elemente dargestellt. Die Funktionalität der Website ist aber trotzdem gewährleistet. Wenn Sie diese Website regelmässig benutzen, empfehlen wir Ihnen, auf Ihrem Computer einen aktuellen Browser zu installieren.Zurück zur Einstiegsseite Drucken Grössere Schrift', '')
        text = text.replace('Vorwärts ähnliche Leitentscheide suchenähnliche Urteile ab 2000 suchen Drucken nach oben', '')
        text = text.replace('Bundesgericht Tribunal fédéral Tribunale federale Tribunal federal', '')
        text = text.replace('Navigation Neue Suche Zurück zum Suchresultat Rang: Zurück 180', '')
        text = text.replace('Navigation Neue Suche Zurück zum Suchresultat Rang:1 ähnliche Leitentscheide suchenähnliche Urteile ab 2000 suchen Drucken nach oben', '')
    
        long_str = long_str + text
        
    df_new = pd.DataFrame(pd.Series( long_str[ix:ix+500] for ix in list(range(len(long_str))) ).value_counts()).reset_index()
    df_new = df_new[df_new[0]> 1]
    
    frames = [df_, df_new]
    df_ = pd.concat(frames)
    

 87% (7 of 8) |########################    | Elapsed Time: 0:07:57 ETA: 0:01:08

# Deleting Duplicates

In [6]:
df_.columns = [['String', 'Count']]

In [7]:
df_ = df_.drop_duplicates(subset='String', keep='first')

In [8]:
df_.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2171792 entries, 0 to 311083
Data columns (total 2 columns):
String    object
Count     float64
dtypes: float64(1), object(1)
memory usage: 49.7+ MB


In [9]:
df = df_[df_['Count'] >= 10]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28359 entries, 0 to 3860
Data columns (total 2 columns):
String    28359 non-null object
Count     28359 non-null float64
dtypes: float64(1), object(1)
memory usage: 664.7+ KB


# Matching similar Strings
Using Jellyfish and remodelling the original df, using it to create a new list of string. Because these ones are the ones I don't need to consider anymore.

In [ ]:
df_new = pd.DataFrame(columns=['String', 'Count', 'Jaro'])

for count in range(1000):
    
    print(count)
    string_list = list(df['String'])
    value_list = []
    bar = progressbar.ProgressBar()

    for elem, i in zip(string_list, bar(range(len(string_list)))):
    
        value = jellyfish.jaro_distance(string_list[0], elem)
        value_list.append(value)
    
    df['Jaro'] = value_list
    df_unique = df[df['Jaro'] >= .825].head(1)

    frames = [df_new, df_unique]
    df_new = pd.concat(frames)

    df = df[df['Jaro'] <= .825]
    del df['Jaro']

  3% (1077 of 28359) |                     | Elapsed Time: 0:00:00 ETA: 0:00:04

0


 98% (28001 of 28359) |################### | Elapsed Time: 0:00:04 ETA: 0:00:00/Users/barneyjs/.virtualenvs/master/lib/python3.5/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
  5% (1434 of 28311) |#                    | Elapsed Time: 0:00:00 ETA: 0:00:03

1


  3% (1074 of 28263) |                     | Elapsed Time: 0:00:00 ETA: 0:00:03

...

  5% (892 of 17608) |#                     | Elapsed Time: 0:00:00 ETA: 0:00:02

245


  5% (884 of 17445) |#                     | Elapsed Time: 0:00:00 ETA: 0:00:02

246


 20% (3534 of 17445) |####                 | Elapsed Time: 0:00:00 ETA: 0:00:02

# Cleaning the New DF and Creating a String

In [ ]:
df_new.info()

In [ ]:
del df_new['Count']

In [ ]:
del df_new['Jaro']

In [ ]:
len(list(df_new['String']))

In [ ]:
string_list = list(df_new['String'])

# Iterating through all the files

Functions to pull out relevant information

In [ ]:
REGEXES_SCHREIBER = [
    r'[gG]reffier\s*:*\s*\w*\.*\s*\w*',
    r'[Gg]reffière\s*:*\s*\w*\.*\s*\w*',
    r'[gG]erichtsschreiberi*n*\s*:* \w*\.*\s*\w*',
    r'[Cc]ancelliere*n*\s*:* \w*\.*\s*\w*']

In [ ]:
def gerichtsschreiber(doc):
    try:
        text = next(chain.from_iterable(re.finditer(r, doc) for r in REGEXES_SCHREIBER), None)
        return text.group()
    except:
        None

In [ ]:
def extracting_date(doc):
    Datum = re.findall(r"[0-9]+\.[0-9]+\.20[0-9]+", doc)
    try:
        return Datum[0]
    except:
        pass

In [ ]:
def extracting_akten_nummer(doc):
    an = re.findall(r"[0-9]*[A-Z][.]*[_]*[ ]*[0-9]+/[0-9]+", doc)
    try:
        an = an[0]
        an = an.replace('_', ' ')
        return an
    except:
        pass

In [ ]:
l = list(sozrecht)

In [ ]:
txt_elems = []
bar = progressbar.ProgressBar()

for text_file, i in zip(l, bar((range(len(l))))):
    
    file = open('txt_files/' + text_file, 'r')
    text = file.read()
    file.close()
    
    #Special formatting
    text = text.replace('\n', ' ').replace('\xa0', '').replace('  ', '')
    text = text.replace('  ', ' ')
    
    Datum = extracting_date(text)
    Aktennummer = extracting_akten_nummer(text)
    Gerichtsschreiber = gerichtsschreiber(text)
    
    
    if string_list[0] in text:
        boolean = 1
    else:
        boolean = 0
    
    mini_dict = {'Text Nummer': text_file,
                 'Datum': Datum,
                 string_list[0][:20]: boolean,
                 'Aktennummer': Aktennummer,
                 'Gerichtsschreiber': Gerichtsschreiber}
    
    txt_elems.append(mini_dict)

In [ ]:
df_bools = pd.DataFrame(txt_elems)

# Repeat what I did above, now attaching the lists to df_bools

In [ ]:
bar = progressbar.ProgressBar()

for string, i in zip(string_list, bar(range(len(string_list)))):
    
    bool_list = []
    
    for text_file in l:
    
        file = open('txt_files/' + text_file, 'r')
        text = file.read()
        file.close()
    
        #Special formatting
        text = text.replace('\n', ' ').replace('\xa0', '').replace('  ', '')
        text = text.replace('  ', ' ')
    
        if string in text:
            boolean = 1
        else:
            boolean = 0
        
        bool_list.append(boolean)
    
    df_bools[string[:20]] = bool_list

In [ ]:
df_bools.info()

In [ ]:
df_bools.to_csv('bools900text_snippets_fr_zivil.csv')

In [ ]:
df_bools = pd.read_csv('bools900text_snippets_fr_zivil.csv')

In [ ]:
del df_bools['Unnamed: 0']

# Making Total Column

In [ ]:
df_bools['TOTAL'] = df_bools.sum(axis=1, numeric_only=True)

In [ ]:
df_bools['TOTAL'].describe()

# Dates

In [ ]:
df_bools['Datetime'] = pd.to_datetime(df_bools['Datum'], format="%d.%m.%Y")
df_bools.index = df_bools['Datetime']

# Plotting and Linear Regression

In [ ]:
df_counts_pyear = pd.DataFrame(df_bools.resample('A')['TOTAL'].sum()).reset_index()

In [ ]:
def change(elem):
    elem = str(elem)
    elem = int(elem.replace('-12-31 00:00:00', ''))
    return elem

In [ ]:
df_counts_pyear

In [ ]:
df_counts_pyear['Years'] = df_counts_pyear['Datetime'].apply(change)

In [ ]:
lm = smf.ols(formula="TOTAL~Years",data=df_counts_pyear).fit()

In [ ]:
lm.params

In [ ]:
intercept, slope = lm.params

In [ ]:
df_counts_pyear.plot(x="Years",y="TOTAL")
plt.plot(df_counts_pyear["Years"],slope*df_counts_pyear["Years"]+intercept,"-",color="red") 

# Without 2017

In [ ]:
df_2007_2016 = df_counts_pyear[df_counts_pyear['Years'] != 2017]

In [ ]:
lm = smf.ols(formula="TOTAL~Years",data=df_2007_2016).fit()

In [ ]:
lm.params

In [ ]:
intercept, slope = lm.params

In [ ]:
df_2007_2016.plot(x="Years",y="TOTAL")
plt.plot(df_2007_2016["Years"],slope*df_2007_2016["Years"]+intercept,"-",color="red")

In [ ]:
df_2007_2016.to_csv('2007_2016_zivil_fr.csv')

# Harmonising the Gerichtsschreiber

In [ ]:
def harm(elem):
    try:
        elem = elem.replace('Greffier: ', '').replace('Greffière: ', '')
        elem = elem.replace('Greffier : ', '').replace('Greffière : ', '')
        elem = elem.replace('M. ', '').replace('Mme ', '')
        elem = elem.replace('M ', '').replace('Mme ', '')
        return elem
    except:
        None

In [ ]:
df_bools['Schreiber harm'] = df_bools['Gerichtsschreiber'].apply(harm)

In [ ]:
df_bools.info()

In [ ]:
df_bools['Schreiber harm'].value_counts().head()

# Merging with large file to sort out Abgeschrieben/Nichteingetreten

In [ ]:
df = pd.read_csv('data/df_final_harm.csv')
df = df[['Aktennummer', 'Entscheid_harm']].copy()
df_ = df_bools.merge(df, left_on='Aktennummer', right_on='Aktennummer')
df_ = df_[df_['Entscheid_harm'] != 'Nicht eingetreten']

In [ ]:
df_.info()

# Working out, total by Clerk

In [ ]:
df_totalelems = pd.DataFrame(df_.groupby('Schreiber harm')['TOTAL'].sum().sort_values(ascending=False))

# Working out number of appeals, we counted their involvement in

In [ ]:
df = pd.read_csv('data/df_final_harm.csv')
df = df[df['Entscheid_harm'] != 'Nicht eingetreten']

In [ ]:
df_sozrecht = df[df['recht_harm'] == 'Zivilrecht']

In [ ]:
df_sozrecht['Schreiber_harm'] = df_sozrecht['Schreiber_harm'].apply(harm)

In [ ]:
df_totalappeals = pd.DataFrame(df_sozrecht['Schreiber_harm'].value_counts())

# Merging total Elem Count with total Appeals by Clerk

In [ ]:
df_final = df_totalelems.merge(df_totalappeals, how='left', left_index=True, right_index=True)

In [ ]:
df_final.columns = [['ELEM 500 COUNT', 'TOTAL APPEALS']]
df_final['Elem pCase'] = round(df_final['ELEM 500 COUNT'] / df_final['TOTAL APPEALS'], 1)

# Gutheissungsquote nach Gerichtsschreiber

In [ ]:
df_gut = pd.read_csv('data/schreiber_percent.csv')

In [ ]:
df_gut = df_gut[['Schreiber', 'Gutgeheissen %']].copy()
def remove(elem):
    elem = elem.replace('.', '')
    return elem
df_gut['Schreiber'] = df_gut['Schreiber'].apply(remove)
df_gut.index = df_gut['Schreiber']
del df_gut['Schreiber']

# Merge with final df

In [ ]:
df_final = df_final.merge(df_gut, how='left', left_index=True, right_index=True)

In [ ]:
df_final.sort_values(by='Elem pCase', ascending=False).head(50)
df_final = df_final[df_final['TOTAL APPEALS'] > 30]

In [ ]:
#df_final.drop('von', inplace=True)

In [ ]:
df_final.columns = [['ELEM 500 COUNT', 'TOTAL APPEALS', 'ElempCase', 'Gutgeheissen']]

In [ ]:
df_final.to_csv('zivil_500_fr.csv')

# Is there a correlation between amout of copy pasted elements and the percentage of successful appeals?

In [ ]:
df_final.plot(kind='scatter', x='ElempCase', y='Gutgeheissen')

In [ ]:
lm = smf.ols(formula="ElempCase~Gutgeheissen",data=df_final).fit()

In [ ]:
intercept, slope = lm.params

In [ ]:
lm.params

In [ ]:
df_final.plot(kind='scatter', x="Gutgeheissen",y="ElempCase")
plt.plot(df_final["Gutgeheissen"],slope*df_final["Gutgeheissen"]+intercept,"-",color="red") 

In [ ]:
df_final

In [ ]:
df_final.to_csv('zivil_500_fr.csv')